# 練習網址：http://www.chinabooktrading.com/song/
## 目標
爬取全部宋詞之作者名、押韻、體詩、及詩等。
## 步驟
1. 設定requests。
2. 設定user-agent池，以便替換。
3. 爬蟲。

In [1]:
#!/usr/bin/env python
#coding=utf-8

In [2]:
# 建立各種 HTTP 請求，從網頁伺服器上取得想要的資料
import requests

# 用來解析處理requests取得的數據; processing XML and HTML
#from lxml import html

# 一個HTML解析器，可以處理各種奇怪的HTML
from bs4 import BeautifulSoup as bs

# 睡眠用
from time import sleep

import json

# 能處理載入、整理與視覺化等常見的資料應用套件
import pandas as pd

# 隨機睡眠用
import random

import os

#偽造User-Agent
from fake_useragent import UserAgent

print('套件載入完成')

套件載入完成


### 設定requests

In [3]:
s = requests.Session()

# 得cookies
# 發現從左側宋詞列表按右鍵，"複製連結“得超連結網址
res = s.get('http://www.chinabooktrading.com/song/fullText.jsp?a=1&e=1&u=1&b=4')
cookies = s.cookies.get_dict()
# print(s.cookies.get_dict())

In [4]:
# 解析網頁
html = bs(res.text)
print(html)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="zh-CN" xml:lang="zh-CN" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="poem" http-equiv="Description"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>poem</title>
<link href="static/fulltext.css" rel="stylesheet" type="text/css"/>
<script type="text/javascript">
			var isGlVisible = false;
			var $ = function(id) {
				return document.getElementById(id);
			};
			var switchGl = function(id) {
				if (isGlVisible == true) {
					isGlVisible = false;
					if (id !== "btngl3") {
						$("btngl1").innerHTML = "顯示格律";
						$("btngl2").innerHTML = "顯示格律";
					} else {
						$("btngl3").innerHTML = "顯示格律";
					}
					var elems = document.getElementsByTagName("p");
					for (var i=0; i<elems.length; i++) {
						if (elems[i].className == "gl") {
							elems[i].style.display = "none";
						}
					}
				} else {
				

In [5]:
# 縮小範圍
html = html.find("div")
html

<div class="content">
<div class="content">
<h2 class="nametitle subtitle TEXT_MEDIUM">詩:宋詩  日詩   (宋太祖)</h2>
<div class="viewopts">
                            
                                【<a class="" href="javascript://;" id="btngl3" onclick="switchGl('btngl3');">顯示格律</a>】
                                【<a class="" href="resultList.jsp?e=1&amp;a=1&amp;b=1&amp;v=7&amp;x=00&amp;s=%E6%9C%88" target="sider">查與本詩形式相似詩</a>】
                            
                        </div>
<div style="clear: both; margin-bottom: 25px;">
<h3 class="TEXT_DARK"> 日詩   </h3>
<h4 class="TEXT_LIGHT">宋太祖</h4>
<h4 class="TEXT_LIGHT">押六月</h4>
<h4 class="TEXT_LIGHT">古體詩</h4>
<p>欲出未出光辣達，千山萬山如火發。</p>
<p class="gl">仄仄仄仄平仄仄，平平仄平○仄仄。</p>
<p>須臾走向天上來，逐却殘星趕却月。</p>
<p class="gl">平平仄仄平仄平，仄仄平平？仄仄。</p>
</div>
<h2 class="subtitle TEXT_MEDIUM">帶註釋文本</h2>
<div>
<h3 class="TEXT_DARK"> 日詩   </h3>
<p>欲出未出光辣達，<span class="comment">《庚溪詩話》作太陽初出光赫赫</span>千山萬山如火發。須臾走向天上來，<span class="comment">《庚溪詩話》作一輪頃刻上天衢</span>逐却殘星趕却<spa

### 爬蟲

#### 1. 風險規避，定時存擋

In [6]:
## 風險規避，定時存檔
def save():
    print("-" * 80)
    print("**定時存檔中...") # 檢查哨
    dn = "ouptput" # 設立output資料夾
    file = "SongShi_content_" + str(i) # 宋詞檔名 # i == 資料筆數
    error_file = "error_list" # 錯誤連結清單檔名

    try:
        if not os.path.exists(dn):   
            os.makedirs(dn)
    except:
        pass

    # 存json格式
    ## 宋詞
    f = open(dn + "/"+ file + ".json", "w", encoding="utf-8") # 打開路徑
    json.dump(final_dic, f) # 寫入json格式
    f.close() # 關閉路徑
    
    ## 錯誤清單
    e = open(dn + "/"+ error_file + ".json", "w", encoding="utf-8") # 打開路徑
    json.dump(error_list, e) # 寫入json格式
    e.close() # 關閉路徑
    print("json存檔完成") # 檢查哨

    # 存csv格式
    ## 宋詞
    final_dic_df = pd.DataFrame(final_dic)
    final_dic_df.to_csv(dn + "/"+ file + ".csv",encoding="utf-8", index=False)
    
    ## 錯誤清單
    error_list_df = pd.DataFrame(error_list)
    error_list_df.to_csv(dn + "/"+ error_file + ".csv",encoding="utf-8", index=False)
    print("csv存檔完成") # 檢查哨
    print("-" * 80)
    print("-" * 80)

    # DONE
    print("存檔完成") # 檢查哨

#### 2. 爬蟲
經測試後，發現替換user-agent即可

In [7]:
# user-agent池
UA_pool = [] #空池
ua = UserAgent() # 設定User-Agent()

for i in range(50): # 先取50個偽User-Agent就好
    UA_pool.append(ua.random) # 偽User-Agent加入UA_pool

# 隨機生成UA的函式
def UA():  
    num = random.randint(0, 20) 
    UA = UA_pool[num]
    headers = {'User-Agent': UA} #ua.random
    return headers

In [8]:
# 設個最終字串，供爬完後放入

final_dic = [] # 總字典
error_list = [] #錯誤回報

In [ ]:
# start 設定
try: 
    start_ = final_dic[-1]['id'] + 1 # 接續上一筆資料'後一筆'開始爬，所以須+1
    print('***接續上一筆資料後開始爬, 開始id: ', start_)
except:
    start_ = 1 # 如果id沒有值，則設為1
    print("***找無id值,從id: ", start_, "開始")

# end設定    
end_ = 254240+1

print("="*80)
print("")

# 開始爬蟲
for i in range(start_, end_):

    URL = 'http://www.chinabooktrading.com/song/fullText.jsp?a=1&e=1&u='+str(i)+'&b=4'
    print("抓取網址: ", URL) # 檢查哨
    headers = UA()
    print("偽user-agent: ", headers)

    # 發現從左側宋詞列表按右鍵，"複製連結“得超連結網址
    res = s.get(URL, headers = headers) #, timeout= 10) # 詩連結
    
    try:
        # 解析網頁
        html = bs(res.text)
    
        # 取詞體
        poem  = html.find("div", style = "clear: both; margin-bottom: 25px;")
        print("解析網頁...") # 檢查哨

        # 要存入的內容
        title = poem.find("h3", class_ = "TEXT_DARK").text # 詞名
        author = poem.find_all("h4", class_ = "TEXT_LIGHT")[0].text # 作者名
        rhyme = poem.find_all("h4", class_ = "TEXT_LIGHT")[1].text # 押韻
        poemType = poem.find_all("h4", class_ = "TEXT_LIGHT")[2].text # 體詩

        # 詩內容
        content =  poem.find_all("p", class_ = "")
        # 將詩list合併成字串
        text = ""  # str
        for x in range (len(content)):
            text += content[x].text

        poemCount = html.find_all("h4", class_ = "TEXT_LIGHT")[-1].text # 作者總共做了幾首詩
        history = html.find_all("p")[-1].text # 作者的生平故事
        print("id =", i, ", 「", title, "」參數設定完成") # 檢查哨


        # 創詞字典
        dic= {
            "id": i,
            "詞名": title,
            "作者": author,
            "押韻": rhyme,
            "體詩": poemType,
            "內容": text,
            "作者共做了幾首詩": poemCount,
            "作者生平": history,
            "網址": URL
             }

        # 加入總字典
        final_dic.append(dic)
        print("加入總字典完成") # 檢查哨
        print("-" * 80)

        # 隨機秒數休眠
        #sleepTime = random.randint(0, 1) 
        #sleep(sleepTime)
        
        if i % 50000 ==0: # 每5萬筆存一次
            save()
            
        
    except:
        # 將錯誤連結網址建成字典
        error_dic = {
            "id":i,
            "網址":URL
        }
        print("***錯誤連結id: ", i, ", 網址: ", URL)
        
        # 將「錯誤連結字典」放入「錯誤連結清單」
        error_list.append(error_dic )
        print("***錯誤連結加入錯誤清單完成")
        print("-" * 80)

save() # 總存檔

print("爬蟲Done!")

In [28]:
# 檢視最後一筆資料
final_dic[-1]

{'id': 207933,
 '詞名': ' 曲肱詩  其一三 ',
 '作者': '白玉蟾',
 '押韻': '押十灰',
 '體詩': '七言絕句\u3000首句平起入平韻式',
 '內容': '白雲隨我見天台，又趁金華路上迴。棲鳳亭中留不去，武夷山下野猿哀。',
 '作者共做了幾首詩': '共有詩1205首',
 '作者生平': '白玉蟾（一一九四～？），本名葛長庚，因繼雷州白氏爲後，改今名。字白叟、以閲、衆甫，號海瓊子、海南翁、瓊山道人、蠙菴、武夷散人、神霄散吏、紫清真人，閩清（今屬福建）人，生于瓊山（今屬海南）。師事陳楠學道，遍歷名山。寧宗嘉定中詔赴闕，命館太乙宮，賜號紫清明道真人（明嘉靖《建寧府志》卷二一）。全真教尊爲南五祖之一。有《海瓊集》、《武夷集》、《上清集》、《玉隆集》等，由其徒彭耜合纂爲《海瓊玉蟾先生文集》四十卷。事見本集卷首彭耜《海瓊玉蟾先生事實》。\u3000白玉蟾詩，以明正統{月翟}仙重編《海瓊玉蟾先生文集》六卷、續集二卷爲底本。校以影印《道藏》本《上清集》、《武夷集》、《玉隆集》（簡稱上清集、武夷集、玉隆集），明萬曆藍格鈔《海瓊白真人文集》（簡稱明鈔本，藏北京大學圖書館），劉雙松安正堂刊《新刻瓊琯白先生集》（簡稱劉本），清乾隆刊《宋海瓊白真人詩文全集》（簡稱乾隆本）。校本多出底本之詩及新輯集外詩，另編爲一卷。',
 '網址': 'http://www.chinabooktrading.com/song/fullText.jsp?a=1&e=1&u=207933&b=4'}

### 萬一爬蟲中斷了...
載入上次中斷地方的json檔

In [14]:
# 宋詞資料
with open("./ouptput/SongShi_content_254240.json",'r') as data: # 開啟資料夾
    final_dic= json.load(data)
data.close() # 關閉資料夾

In [15]:
# 檢視最後一筆資料
final_dic[-2:]

[{'id': 254239,
  '詞名': ' 宣妙樓   ',
  '作者': '方鴻飛',
  '押韻': '押六麻',
  '體詩': '七言律詩\u3000首句仄起入平韻式',
  '內容': '雲觀烟樓是梵家，竹圍如洗逼寒沙。因風綠浪摇晴麥，遇雨紅香落澗花。人鎖晝房聽鳥語，僧歸晚塢放蜂衙。不須老遠來沽酒，只覓天酥爲點茶。',
  '作者共做了幾首詩': '共有詩1首',
  '作者生平': '方鴻飛，嵊縣（今浙江嵊州）人。',
  '網址': 'http://www.chinabooktrading.com/song/fullText.jsp?a=1&e=1&u=254239&b=4'},
 {'id': 254240,
  '詞名': ' 登凌雲山絕頂   ',
  '作者': '葉升',
  '押韻': '押八齊',
  '體詩': '七言律詩\u3000首句平起入平韻式',
  '內容': '崔巍萬仞儼危梯，躡履多憑一杖藜。老宦躋崖沖斗動，高朋立頂惹風啼。振衣遠望乾坤小，舉首回看日月低。自是登天疑有路，雲山縹緲夕陽西。',
  '作者共做了幾首詩': '共有詩1首',
  '作者生平': '葉升，南安（今福建南安東）人。第進士，曾官主簿（《南安縣志》卷四八）。',
  '網址': 'http://www.chinabooktrading.com/song/fullText.jsp?a=1&e=1&u=254240&b=4'}]

In [16]:
# 錯誤連結資料
with open("./ouptput/error_list1.json",'r') as error: # 開啟資料夾
    error_list= json.load(error) 
error.close() # 關閉資料夾
error_list

[{'id': 31791,
  '網址': 'http://www.chinabooktrading.com/song/fullText.jsp?a=1&e=1&u=31791&b=4'},
 {'id': 99531,
  '網址': 'http://www.chinabooktrading.com/song/fullText.jsp?a=1&e=1&u=99531&b=4'}]